In [1]:
pip install openai==0.28

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install chardet

Note: you may need to restart the kernel to use updated packages.


In [5]:
import ipywidgets as widgets
from IPython.display import display, FileLink
import chardet

def parse_ris_file(input_file):
    # Initialize lists to store multiple titles and abstracts
    titles = []
    abstracts = []

    # Open the input file and read line by line
    for line in input_file.split('\n'):
        if line.startswith('T1  - '):
            title = line[6:].strip()
            titles.append(title)
        elif line.startswith('AB  - '):
            abstract = line[6:].strip()
            abstracts.append(abstract)
    
    # Create output text with titles and abstracts
    output_text = ""
    for i in range(max(len(titles), len(abstracts))):
        if i < len(titles):
            output_text += f"Title {i + 1}: {titles[i]}\n"
        if i < len(abstracts):
            output_text += f"Abstract {i + 1}: {abstracts[i]}\n"
        output_text += "\n"

    return output_text

def on_parse_ris_button_click(b):
    # Get uploaded file information
    uploaded_file = ris_file_upload.value[0]
    raw_data = bytes(uploaded_file['content'])  # Convert memoryview to bytes
    
    # Detect the file encoding
    result = chardet.detect(raw_data)
    encoding = result['encoding']
    
    # Decode the file content using the detected encoding
    file_content = raw_data.decode(encoding)
    
    # Parse the RIS file content
    parsed_content = parse_ris_file(file_content)
    
    # Save parsed content to output.txt
    with open('parsed_output.txt', 'w') as file:
        file.write(parsed_content)
    
    # Display the download link for the parsed file
    with upload_output:
        upload_output.clear_output()
        display(FileLink('parsed_output.txt', result_html_prefix="Click here to download the parsed text file: ", result_html_suffix=""))

# Setup interactive widgets for RIS file upload and processing
ris_file_upload = widgets.FileUpload(
    accept='.ris',  # Accepted file formats
    multiple=False  # Single file upload
)

parse_ris_button = widgets.Button(
    description='Parse RIS Texts',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to parse RIS file',
    icon='file-import' 
)

upload_output = widgets.Output()

# Bind the button click event
parse_ris_button.on_click(on_parse_ris_button_click)

# Display widgets
display(
    widgets.VBox([
        ris_file_upload,
        parse_ris_button,
        upload_output
    ])
)


In [13]:
import openai
import os
from IPython.display import display, HTML
import ipywidgets as widgets
import chardet

# Setup OpenAI API Key
openai.api_key = "insert-your-key-here"

def generate_search_strings(objective, questions):
    prompt = f"""
    I am conducting a scoping/literature review with the objective of {objective} and have the following questions in particular: {questions}

    Based on these questions, can you help me generate potential search strings to find relevant papers and documents for this literature review? Please provide search strings that cover various aspects of these questions.
    """

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=3000,
            n=1,
            temperature=0.5,
        )
        return response['choices'][0]['message']['content']
    except openai.error.OpenAIError as e:
        return f"An error occurred: {str(e)}"

def review_abstracts(objective, questions, additional_info, input_text, selection):
    additional_info_str = f" Here are some additional instructions or pointers: {additional_info}" if additional_info else ""
    
    prompts = {
        "zero_shot": f"I am conducting a scoping/literature review with the objective of {objective} and have the following questions in particular: {questions}.{additional_info_str} Based on the following abstracts provided, should I include them in my literature review? For each abstract, respond with 'yes' or 'no' for inclusion, and format the output as 'Title | Yes/No':\n{input_text}\n\n",
        "cot": f"""
I am conducting a scoping/literature review with the objective of {objective} and have the following questions in particular: {questions}.{additional_info_str} Based on the following abstracts provided, should I include each of them in my literature review? For each abstract, respond with 'yes' or 'no' for inclusion, and format the output as 'Title | Yes/No':

{input_text}

Here's an example evaluation:
- Abstract: "Impacts of education on learning techniques"
- Step 1: Aligns with objective? Yes, it focuses on learning techniques.
- Step 2: Addresses specific questions? Yes, it looks into educational impacts.
- Step 3: Methodology sound? Yes, it uses robust data analysis.
- Step 4: Results significant? Yes, it provides applicable insights.
- Decision: "Impacts of education on learning techniques | Yes"
""",
        "prompt_chaining": f"""
I am conducting a scoping/literature review with the objective of {objective} and have the following questions in particular: {questions}.{additional_info_str} Based on the following abstracts provided, should I include each of them in my literature review? For each abstract, respond with 'yes' or 'no' for inclusion, and format the output as 'Title | Yes/No':

{input_text}

To guide your assessment, evaluate each abstract individually. For example, consider an abstract titled 'Impacts of education on learning techniques'. Would you include this in your literature review? Provide a response like 'Impacts of education on learning techniques | Yes' or 'Impacts of education on learning techniques | No' along with a brief explanation."
""",
        "combo": f"""
I am conducting a scoping/literature review with the objective of {objective} and have the following questions in particular: {questions}.{additional_info_str} Based on the following abstracts provided, should I include each of them in my literature review? For each abstract, respond with 'yes' or 'no' for inclusion, and format the output as 'Title | Yes/No':

{input_text}

To help with your decision, follow these steps for each abstract:
1. Determine if the abstract aligns with the review objective.
2. Assess if it addresses any of the specific questions.
3. Evaluate the study's methodology.
4. Consider the significance and applicability of the results.
5. Summarize your thought process and provide a final decision.

Here's an example evaluation:
- Abstract: "Impacts of education on learning techniques"
- Step 1: Aligns with objective? Yes, it focuses on learning techniques.
- Step 2: Addresses specific questions? Yes, it looks into educational impacts.
- Step 3: Methodology sound? Yes, it uses robust data analysis.
- Step 4: Results significant? Yes, it provides applicable insights.
- Decision: "Impacts of education on learning techniques | Yes"
"""
    }

    prompt = prompts[selection]

    try:
        # First call to ChatGPT
        response1 = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=3000,
            n=1,
            temperature=0.5,
        )
        review1 = response1['choices'][0]['message']['content']

        # Second call to ChatGPT
        response2 = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=3000,
            n=1,
            temperature=0.5,
        )
        review2 = response2['choices'][0]['message']['content']

        # Third call to ChatGPT to summarize results
        summary_prompt = f"""
        Here are the results of two reviews for the same set of abstracts. For every single abstract in each response, provide a final decision based on the following rules:
        - If both reviews give 'yes', the final decision is 'yes'.
        - If both reviews give 'no', the final decision is 'no'.
        - If one review gives 'yes' and the other gives 'no', the final decision is 'maybe'.

        Review 1:
        {review1}

        Review 2:
        {review2}

        Format the output as a table with columns: Title, Final Decision.
        """

        response3 = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": summary_prompt}
            ],
            max_tokens=3000,
            n=1,
            temperature=0.5,
        )
        summary = response3['choices'][0]['message']['content']

        return review1, review2, summary
    except openai.error.OpenAIError as e:
        return f"An error occurred: {str(e)}"

# Setup interactive widgets
objective_text = widgets.Textarea(
    value='Understand the current landscape of learning analytics (LA) and educational data mining (EDM) in post-secondary business education.',
    placeholder='Enter the objective of your review',
    description='Objective:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='80px')
)

questions_text = widgets.Textarea(
    value= "What are the typical learning environments and characteristics used to study the impact of learning analytics in post-secondary business education?, " \
            "What are the overarching conclusions from systematic and scoping reviews on the state of learning analytics in post-secondary education, with a focus on business education?," \
            "How can these results be leveraged to successfully incorporate learning analytics in post-secondary pedagogical design in a business school?, " \
            "What role do machine learning and other predictive tools have in developing intervention strategies in post-secondary business education?",
    placeholder='Enter your review questions',
    description='Questions:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='80px')
)

additional_info_text = widgets.Textarea(
    value='',
    placeholder='Enter any additional information or instructions',
    description='Additional Info:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='80px')
)

input_text = widgets.Textarea(
    value='',
    placeholder='Enter abstracts to review',
    description='Abstracts:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='200px')
)

file_upload = widgets.FileUpload(
    accept='.txt,.csv',
    multiple=False
)

selection_radio = widgets.RadioButtons(
    options=['zero_shot', 'cot', 'prompt_chaining', 'combo'],
    value='zero_shot',
    description='Prompting Option:',
    disabled=False
)

generate_button = widgets.Button(
    description='Generate Search Strings',
    disabled=False,
    button_style='',
    tooltip='Click to generate search strings',
    icon='search'
)

review_button = widgets.Button(
    description='Review Abstracts',
    disabled=False,
    button_style='',
    tooltip='Click to review abstracts',
    icon='check'
)

def on_generate_button_clicked(b):
    objective = objective_text.value
    questions = questions_text.value
    search_strings = generate_search_strings(objective, questions)
    display(HTML(f"<h3>Generated Search Strings:</h3><pre>{search_strings}</pre>"))

def on_review_button_clicked(b):
    objective = objective_text.value
    questions = questions_text.value
    additional_info = additional_info_text.value
    input_abstracts = input_text.value
    selection = selection_radio.value
    review1, review2, summary = review_abstracts(objective, questions, additional_info, input_abstracts, selection)
    display(HTML(f"<h3>Review 1:</h3><pre>{review1}</pre>"))
    display(HTML(f"<h3>Review 2:</h3><pre>{review2}</pre>"))
    display(HTML(f"<h3>Summary:</h3><pre>{summary}</pre>"))

generate_button.on_click(on_generate_button_clicked)
review_button.on_click(on_review_button_clicked)

# Display widgets
display(HTML("<h1>Scoping/Literature Review Tool</h1>"))
display(objective_text)
display(questions_text)
display(additional_info_text)
display(input_text)
display(file_upload)

display(selection_radio)
display(generate_button)
display(review_button)


Textarea(value='Understand the current landscape of learning analytics (LA) and educational data mining (EDM) …

Textarea(value='What are the typical learning environments and characteristics used to study the impact of lea…

Textarea(value='', description='Additional Info:', layout=Layout(height='80px', width='100%'), placeholder='En…

Textarea(value='', description='Abstracts:', layout=Layout(height='200px', width='100%'), placeholder='Enter a…

FileUpload(value=(), accept='.txt,.csv', description='Upload')

RadioButtons(description='Prompting Option:', options=('zero_shot', 'cot', 'prompt_chaining', 'combo'), value=…

Button(description='Generate Search Strings', icon='search', style=ButtonStyle(), tooltip='Click to generate s…

Button(description='Review Abstracts', icon='check', style=ButtonStyle(), tooltip='Click to review abstracts')